In [14]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import AutoModel , AutoImageProcessor , AutoTokenizer 
from datasets import load_dataset , Dataset
from datasets import Image as dImage
from PIL import Image
import torch
from torch import nn

torch.cuda.empty_cache()

# import os
# os.environ['TF_GPU_ALLOCATOR']='cuda_malloc_async'

2024-05-16 12:42:02.774786: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-16 12:42:03.349117: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/user/miniconda3/envs/python38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
df_acc = pd.read_pickle("./vqa_v2_acc.pkl")
df_acc.head()

,question,question_type,question_id,image_id,answer_type,label
0,What is this photo taken looking through?,what is this,458752000,COCO_train2014_000000458752.jpg,other,"{'ids': ['net', 'netting', 'mesh'], 'weights':..."
1,What position is this man playing?,what,458752001,COCO_train2014_000000458752.jpg,other,"{'ids': ['pitcher', 'catcher'], 'weights': [0...."
2,What color is the players shirt?,what color is the,458752002,COCO_train2014_000000458752.jpg,other,"{'ids': ['orange'], 'weights': [1.0]}"
3,Is this man a professional baseball player?,is this,458752003,COCO_train2014_000000458752.jpg,yes/no,"{'ids': ['yes', 'no'], 'weights': [0.94, 0.06]}"
4,What color is the snow?,what color is the,262146000,COCO_train2014_000000262146.jpg,other,"{'ids': ['white'], 'weights': [1.0]}"


In [29]:
np.array(Image.open("./train2014_3d/" + df_acc["image_id"][0])).shape

(480, 640, 3)

In [60]:
# SAMPLES = 100
# df = df_acc[:SAMPLES]

df = df_acc.groupby('question_type').sample(n=800 , replace=True)
df = df.reset_index(drop=True)
SAMPLES = len(df)

print("Number of samples are : " , SAMPLES)

df.head()

Number of samples are :  52000


,question,question_type,question_id,image_id,answer_type,label
0,Are all the seats filled?,are,194694017,COCO_train2014_000000194694.jpg,yes/no,"{'ids': ['yes', 'no', 'no seats'], 'weights': ..."
1,Are most people sitting or standing?,are,106836005,COCO_train2014_000000106836.jpg,other,"{'ids': ['sitting', 'standing'], 'weights': [0..."
2,Are both of the sheep facing in the same direc...,are,425398000,COCO_train2014_000000425398.jpg,yes/no,"{'ids': ['yes'], 'weights': [1.0]}"
3,Are objects in the picture vases?,are,209967000,COCO_train2014_000000209967.jpg,yes/no,"{'ids': ['no'], 'weights': [1.0]}"
4,Are all the people looking at the same thing?,are,458785002,COCO_train2014_000000458785.jpg,yes/no,"{'ids': ['no', 'yes'], 'weights': [0.88, 0.12]}"


Creating Labels

In [61]:
import itertools

labels = [item['ids'] for item in df['label'].to_list()]
flattened_labels = list(itertools.chain(*labels))
unique_labels = list(set(flattened_labels))

label2id = {label: idx for idx, label in enumerate(unique_labels)}
id2label = {idx: label for label, idx in label2id.items()} 

In [62]:
label2id

{'yellow, white and purple': 0,
 'near people': 1,
 '0505': 2,
 'initech': 3,
 'skiing in winter': 4,
 '2:43': 5,
 "it's close": 6,
 '64+': 7,
 'gibson city': 8,
 'to keep sand off of them': 9,
 'hot dog and cheese': 10,
 "one that can't be seen": 11,
 'pulling his tie': 12,
 'curtain and bright sun': 13,
 'ball catcher': 14,
 'yacht': 15,
 'in bushes': 16,
 '163 080-5': 17,
 'to pick up guy': 18,
 'pants and dress': 19,
 '11:55': 20,
 'crew cut': 21,
 'coaster': 22,
 '2nd 1 from left': 23,
 'yes white dressing': 24,
 'mirror, teeth': 25,
 'cans': 26,
 'outhouse': 27,
 'chethams school of medicine': 28,
 'parrots': 29,
 'red, green, brown-white': 30,
 'mulberry st': 31,
 'loading trash into truck': 32,
 "they're under umbrella": 33,
 '2 days': 34,
 'angst': 35,
 'bath shower hot cold wet water clean': 36,
 'two girls': 37,
 'move forward': 38,
 'child care items': 39,
 'there is no space to store it': 40,
 'on top snow': 41,
 'use it': 42,
 'holiday inn': 43,
 'rockies': 44,
 'cutting 

In [63]:
NUM_CLASSES = len(id2label)
print("Number of Classes : " , NUM_CLASSES)

Number of Classes :  33078


In [64]:
n = SAMPLES
rf = 10 ** (len(str(n))-1)
r = str(int(np.round(n / rf) * rf))

r = r[:-3] + r[-3:].replace("000" , "k")

pd.to_pickle(df , f"./vqa_v2_sampled_{r}.pkl")